In [1]:
from keras import backend as K
from keras.layers import Input, Dense, Lambda, Activation, Dropout, BatchNormalization, Layer
from keras.models import Model
from keras.optimizers import Adam
from keras.datasets import reuters
from keras.callbacks import EarlyStopping
import numpy as np

In [18]:
reuters.load_data?

In [25]:
# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

V = 10922
(X_train, _), (_, _) = reuters.load_data(start_char=None, 
                                         oov_char=None, index_from=-1, 
                                         num_words=V) # remove words having freq(q) <= 5
np.load = np_load_old

In [26]:
word_index = reuters.get_word_index()
index2word = {v-1: k for k, v in word_index.items()} # Zero-origin word index

X_train = np.array([np.bincount(doc, minlength=V) for doc in X_train])
X_train = X_train[:8000, :]

In [28]:
num_hidden = 100
num_topic = 20
batch_size = 100
alpha = 1./20

In [29]:
mu1 = np.log(alpha) - 1/num_topic*num_topic*np.log(alpha)
sigma1 = 1./alpha*(1-2./num_topic) + 1/(num_topic**2)*num_topic/alpha
inv_sigma1 = 1./sigma1
log_det_sigma = num_topic*np.log(sigma1)